## Modelo consolidado para datasets de treino e validação
# Versão 2.0 modelo classificador

In [21]:
# Rode apenas se necessario

! pip install -q tensorflow numpy 

In [1]:
import os
# tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


In [6]:

data_base_path = 'C:\\Users\\User\\Downloads\\Imagens\\base_de_dados'
train_path= os.path.join(data_base_path,'treino').replace(os.path.sep,"\\")
val_path=os.path.join(data_base_path,'validacao').replace(os.path.sep,"\\")
print(val_path,train_path,sep='\n')


C:\Users\User\Downloads\Imagens\base_de_dados\validacao
C:\Users\User\Downloads\Imagens\base_de_dados\treino


In [7]:

heigth,width=180,180 #config img
epochs = 20 # qnts epocas ele ira iterar para o train
batch_size = 32 # tamanho dos lotes
learnig_rate=0.0001 # taxa de aprendizagem para o optimizer



In [9]:


# data set treino
dataset_treino = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=(width,heigth),
    shuffle=True,
    batch_size=batch_size,
)

# data set validacao
dataset_validacao = tf.keras.utils.image_dataset_from_directory(
    val_path,
    image_size=(width,heigth),
    shuffle=True,
    batch_size=batch_size,
)


Found 195 files belonging to 4 classes.
Found 46 files belonging to 4 classes.


In [13]:
nomes_class = dataset_treino.class_names
print(nomes_class)
numero_class = len(nomes_class)
print(numero_class)

['ausente', 'escasso', 'mediana', 'uniforme']
4


In [14]:

# Dividindo o conjunto de validação em validação e teste
val_batches = tf.data.experimental.cardinality(dataset_validacao)
dataset_teste = dataset_validacao.take(val_batches // 2)
dataset_validacao = dataset_validacao.skip(val_batches // 2)

# Contando o número de amostras em cada conjunto
num_val_samples = sum(len(batch) for batch in dataset_validacao)
num_test_samples = sum(len(batch) for batch in dataset_teste)

print(f"Número de amostras na validação: {num_val_samples}")
print(f"Número de amostras no teste: {num_test_samples}")


Número de amostras na validação: 2
Número de amostras no teste: 2


In [22]:

from tensorflow.keras.layers import RandomFlip,RandomRotation,RandomZoom

# data argumentation pra evitar overfit
data_argumentation = Sequential([
    RandomFlip('horizontal_and_vertical'),
    RandomRotation(0.2),
    RandomZoom((0.2,0.3))
])

# rescala
rescaling = layers.Rescaling(scale=1./255)

# criacao do modelo
modelo = Sequential([
    data_argumentation,
    rescaling,
    layers.Conv2D(32,(3,3),padding='same',activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64,(3,3),padding='same',activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128,(3,3),padding='same',activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(256,(3,3),padding='same',activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(256,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(numero_class,activation='softmax')
])

# copilacao do modelo 
modelo.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
  )


In [19]:

# resumo do modelo 
modelo.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 180, 180, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 90, 90, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 90, 90, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 45, 45, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 45, 45, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 22, 22, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 11, 11, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 30976)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     7,930,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,958,670 (95.21 MB)

 Trainable params: 8,319,556 (31.74 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,639,114 (63.47 MB)

In [18]:

# treinamento do modelo

history = modelo.fit(
    dataset_treino, validation_data=dataset_validacao, epochs=epochs, 
)               

# Avaliação do modelo no conjunto de teste
test_loss, test_acc = modelo.evaluate(dataset_teste, verbose=2)
print("\nTest accuracy:", test_acc)


Epoch 1/20


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\keras\src\backend\tensorflow\nn.py:609: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 921ms/step - accuracy: 0.2208 - loss: 1.4436 - val_accuracy: 0.5714 - val_loss: 1.0814
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 850ms/step - accuracy: 0.3814 - loss: 1.2238 - val_accuracy: 0.5000 - val_loss: 1.1400
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 820ms/step - accuracy: 0.3229 - loss: 1.2173 - val_accuracy: 0.5000 - val_loss: 1.1221
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 802ms/step - accuracy: 0.3266 - loss: 1.1921 - val_accuracy: 0.5000 - val_loss: 1.0726
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 865ms/step - accuracy: 0.3202 - loss: 1.2014 - val_accuracy: 0.4286 - val_loss: 1.0277
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 867ms/step - accuracy: 0.2957 - loss: 1.1811 - val_accuracy: 0.5714 - val_loss: 1.0281
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 799ms/step - accuracy: 0.3516 - loss: 1.1917 - val_accuracy: 0.5714 - val_loss: 1.0030
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 835ms/step - accuracy: 0.3677 - loss: 1.2030 - val_accuracy: 0.5714 - val_loss: 1.0355
Ep

In [20]:

import numpy as np
from tensorflow.keras.preprocessing import image

# Função para carregar e preprocessar a imagem
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(heigth, width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Adiciona uma dimensão extra para o batch
    img_array = img_array / 255.0  # Normaliza a imagem como feito no treinamento
    return img_array

# Caminho para a imagem que você quer testar
img_path = 'C:\\Users\\User\\Downloads\\Imagens\\base_de_dados\\validacao\\mediana\\m (23).png'

# Preprocessa a imagem
img_array = preprocess_image(img_path)

# Faz a predição
predictions = modelo.predict(img_array)

# Obtém a classe prevista
predicted_class = np.argmax(predictions, axis=1)
predicted_class_label = nomes_class[predicted_class[0]]

print(f"Predição: {predicted_class_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
Predição: escasso
